TODO:
 - multiple time steps

In [2]:
%load_ext autoreload
%autoreload 2
%cd /home/ftei-dsw/projects/weather-gan

/home/ftei-dsw/projects/weather-gan


In [7]:
from src.data import DataModule
import matplotlib.pyplot as plt
import numpy as np
from src.configuration import Config

In [8]:
data = DataModule(Config())
data.setup('fit')
train_loader = data.train_dataloader()
batch = next(iter(train_loader))

In [12]:
batch[0]['input'].shape, batch[0]['target'].shape, batch[0]['target'].type, batch[0]['target'].device

(torch.Size([32, 2, 72, 72]),
 torch.Size([32, 2, 72, 72]),
 <function Tensor.type>,
 device(type='cuda', index=0))

In [ ]:
batch[0]['target'][0,0,:,:]

In [ ]:

batch = next(iter(train_loader))
batch[0]['input'].shape, batch[0]['target'].shape

plt.imshow(batch[0]['target'][0,0,:,:].cpu())
plt.show()
plt.imshow(batch[0]['target'][0,1,:,:].cpu())
plt.show()
batch[0]['target'][0,0,:,:].min(), batch[0]['target'][0,0,:,:].max(), batch[0]['target'][0,:,:,1].min(), batch[0]['target'][0,:,:,1].max()

In [ ]:
plt.imshow(batch[0]['input'][0,0,:,:].cpu())
plt.show()
plt.imshow(batch[0]['input'][0,1,:,:].cpu())
plt.show()
batch[0]['input'][0,:,:,0].min(), batch[0]['input'][0,:,:,0].max(), batch[0]['input'][0,:,:,1].min(), batch[0]['target'][0,:,:,1].max()

In [20]:
import xarray as xr
ds.close()
ds = xr.open_dataset('/home/ftei-dsw/data/weather-gan/datasets/daily_gfdl_historical.nc')

<xarray.Dataset>
Dimensions:        (time: 1, longitude: 144, latitude: 73)
Coordinates:
  * time           (time) object 2000-01-07 12:00:00
  * longitude      (longitude) float64 -180.0 -177.5 -175.0 ... 175.0 177.5
  * latitude       (latitude) float64 -90.0 -87.5 -85.0 -82.5 ... 85.0 87.5 90.0
Data variables:
    precipitation  (time, latitude, longitude) float32 ...
    temperature    (time, latitude, longitude) float32 ...
Attributes:
    CDI:            Climate Data Interface version 1.9.10 (https://mpimet.mpg...
    Conventions:    CF-1.6
    regrid_method:  bilinear
    history:        Tue Apr 05 18:11:38 2022: cdo merge precipitation_daily_g...
    CDO:            Climate Data Operators version 1.9.10 (https://mpimet.mpg...

In [81]:
x = np.ones((10,5,2))
x[:,:,0] = np.log(x[:,:,0] + 0.001) - np.log(0.001)
x[:,:,1]


array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])